Here we import:
- `defaultdict` → helps us create nested dictionaries easily for trie nodes.
- `Counter` → to count suffix occurrences.
- `pprint` → pretty printing dictionaries (useful for debugging).


In [1]:
# Basic imports
from collections import defaultdict, Counter
import pprint


We load all nouns from `brown_nouns.txt`.
- Each line is stripped of whitespace and converted to lowercase.
- We store all words in a list `words`.
- We check the total number of words and print the first 20 for inspection.


In [6]:
# Load the dataset of nouns
with open("brown_nouns.txt", "r") as f:
    words = [line.strip().lower() for line in f if line.strip()]

print("Total words loaded:", len(words))
print("Sample words:", words[:20])


Total words loaded: 202793
Sample words: ['investigation', 'primary', 'election', 'evidence', 'irregularities', 'place', 'jury', 'presentments', 'charge', 'election', 'praise', 'thanks', 'manner', 'election', 'term', 'jury', 'reports', 'irregularities', 'primary', 'handful']


We define a **TrieNode** class:
- `children`: dictionary mapping characters to next nodes.
- `is_end`: marks if this node ends a word.
- `count`: how many words pass through this node (used later for frequency).

We define a **Trie** class with:
- `insert(word)`: inserts a word character by character into the trie.


In [7]:
class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)  # each node points to children
        self.is_end = False                   # marks end of word
        self.count = 0                        # frequency counter

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for ch in word:
            node = node.children[ch]
            node.count += 1
        node.is_end = True


We build two tries:
1. **Prefix Trie** → words inserted left-to-right.
2. **Suffix Trie** → words inserted right-to-left (so suffixes become prefixes).

This allows us to analyze both perspectives when identifying stems/suffixes.


In [8]:
# Build prefix trie (normal)
prefix_trie = Trie()
for word in words:
    prefix_trie.insert(word)

# Build suffix trie (reverse each word before inserting)
suffix_trie = Trie()
for word in words:
    suffix_trie.insert(word[::-1])

print("Tries built successfully!")


Tries built successfully!


We define `find_split`:
- Traverse the trie character by character.
- Track where the **maximum branching** occurs (`max_branching`).
- That index = point where stem ends and suffix begins.
- If using suffix trie, we reverse the logic.


In [9]:
def find_split(word, trie, reverse=False):
    """
    Traverse the trie for given word.
    The node where maximum branching occurs = suffix boundary.
    """
    node = trie.root
    split_index = 0
    max_branching = 0
    
    letters = word[::-1] if reverse else word
    
    for i, ch in enumerate(letters):
        node = node.children.get(ch)
        if not node:
            break
        branching = len(node.children)
        if branching > max_branching:
            max_branching = branching
            split_index = i + 1
    
    if reverse:
        stem = word[:-split_index] if split_index > 0 else word
        suffix = word[-split_index:] if split_index > 0 else ""
    else:
        stem = word[:split_index] if split_index > 0 else word
        suffix = word[split_index:] if split_index > 0 else ""
    
    return stem, suffix


We test our function on the first 50 words:
- For each word, we compute stem+suffix using **Prefix Trie** and **Suffix Trie**.
- We print results in the format:


In [10]:
results = []

for word in words[:50]:  # testing first 50 words
    stem_p, suf_p = find_split(word, prefix_trie, reverse=False)
    stem_s, suf_s = find_split(word, suffix_trie, reverse=True)
    results.append((word, stem_p, suf_p, stem_s, suf_s))

# Display few results
for r in results[:20]:
    print(f"{r[0]} | PrefixTrie: {r[1]}+{r[2]} | SuffixTrie: {r[3]}+{r[4]}")


investigation | PrefixTrie: in+vestigation | SuffixTrie: investigati+on
primary | PrefixTrie: p+rimary | SuffixTrie: primar+y
election | PrefixTrie: e+lection | SuffixTrie: electi+on
evidence | PrefixTrie: e+vidence | SuffixTrie: evidenc+e
irregularities | PrefixTrie: i+rregularities | SuffixTrie: irregularitie+s
place | PrefixTrie: p+lace | SuffixTrie: plac+e
jury | PrefixTrie: ju+ry | SuffixTrie: jur+y
presentments | PrefixTrie: p+resentments | SuffixTrie: presentment+s
charge | PrefixTrie: c+harge | SuffixTrie: charg+e
election | PrefixTrie: e+lection | SuffixTrie: electi+on
praise | PrefixTrie: p+raise | SuffixTrie: prais+e
thanks | PrefixTrie: t+hanks | SuffixTrie: thank+s
manner | PrefixTrie: ma+nner | SuffixTrie: mann+er
election | PrefixTrie: e+lection | SuffixTrie: electi+on
term | PrefixTrie: t+erm | SuffixTrie: ter+m
jury | PrefixTrie: ju+ry | SuffixTrie: jur+y
reports | PrefixTrie: re+ports | SuffixTrie: report+s
irregularities | PrefixTrie: i+rregularities | SuffixTrie: ir

We compute **frequency of suffixes**:
- Use suffix trie splits.
- Count how many times each suffix occurs.
- Convert frequencies into probabilities = `freq / total_words`.
- Print top 10 suffixes with their probability.


In [11]:
suffix_counter = Counter()

for word in words:
    _, suffix = find_split(word, suffix_trie, reverse=True)
    if suffix:
        suffix_counter[suffix] += 1

print("Most common suffixes:")
for suf, freq in suffix_counter.most_common(10):
    prob = freq / len(words)
    print(f"{suf} -> freq={freq}, prob={prob:.4f}")


Most common suffixes:
s -> freq=55539, prob=0.2739
e -> freq=35090, prob=0.1730
t -> freq=19226, prob=0.0948
on -> freq=14811, prob=0.0730
y -> freq=14792, prob=0.0729
er -> freq=8663, prob=0.0427
d -> freq=7965, prob=0.0393
m -> freq=4355, prob=0.0215
ing -> freq=3869, prob=0.0191
k -> freq=3658, prob=0.0180


Compare Prefix vs. Suffix Trie Stemming

In [13]:
# Compare prefix vs. suffix results for all words
comparison_results = []

for word in words:
    stem_p, suf_p = find_split(word, prefix_trie, reverse=False)
    stem_s, suf_s = find_split(word, suffix_trie, reverse=True)
    comparison_results.append((word, stem_p, suf_p, stem_s, suf_s))

# Display a sample of comparisons
print("Word | PrefixTrie (Stem+Suffix) | SuffixTrie (Stem+Suffix)")
print("-"*60)
for r in comparison_results[:30]:
    print(f"{r[0]} | {r[1]}+{r[2]} | {r[3]}+{r[4]}")


Word | PrefixTrie (Stem+Suffix) | SuffixTrie (Stem+Suffix)
------------------------------------------------------------
investigation | in+vestigation | investigati+on
primary | p+rimary | primar+y
election | e+lection | electi+on
evidence | e+vidence | evidenc+e
irregularities | i+rregularities | irregularitie+s
place | p+lace | plac+e
jury | ju+ry | jur+y
presentments | p+resentments | presentment+s
charge | c+harge | charg+e
election | e+lection | electi+on
praise | p+raise | prais+e
thanks | t+hanks | thank+s
manner | ma+nner | mann+er
election | e+lection | electi+on
term | t+erm | ter+m
jury | ju+ry | jur+y
reports | re+ports | report+s
irregularities | i+rregularities | irregularitie+s
primary | p+rimary | primar+y
handful | ha+ndful | handfu+l
reports | re+ports | report+s
jury | ju+ry | jur+y
interest | in+terest | interes+t
election | e+lection | electi+on
number | nu+mber | numb+er
voters | vo+ters | voter+s
size | s+ize | siz+e
city | c+ity | cit+y
jury | ju+ry | jur+y
regi

**Quantitative Analysis**

To measure which trie performs better for stemming:
- We check **suffix frequencies** (already computed).
- We measure **average suffix length** and **diversity**.
- Higher frequency & shorter suffixes indicate better morphological segmentation.


In [15]:
def evaluate_trie(trie, reverse=False):
    suffix_counter = Counter()
    suffix_lengths = []
    
    for word in words:
        _, suffix = find_split(word, trie, reverse=reverse)
        if suffix:
            suffix_counter[suffix] += 1
            suffix_lengths.append(len(suffix))
    
    avg_suffix_len = sum(suffix_lengths)/len(suffix_lengths) if suffix_lengths else 0
    return suffix_counter, avg_suffix_len

# Evaluate prefix trie
prefix_suffix_counter, prefix_avg_len = evaluate_trie(prefix_trie, reverse=False)

# Evaluate suffix trie
suffix_suffix_counter, suffix_avg_len = evaluate_trie(suffix_trie, reverse=True)

print("Prefix Trie Analysis:")
print("Average Prefix Length:", prefix_avg_len)
print("Top 10 prefixes:", prefix_suffix_counter.most_common(10))

print("\nSuffix Trie Analysis:")
print("Average Suffix Length:", suffix_avg_len)
print("Top 10 suffixes:", suffix_suffix_counter.most_common(10))



Prefix Trie Analysis:
Average Prefix Length: 5.153718323610775
Top 10 prefixes: [('y', 1641), ('ime', 1569), ('n', 1298), ('nd', 1071), ('ears', 1027), ('fe', 927), ('eople', 828), ('se', 826), ('en', 792), ('ear', 784)]

Suffix Trie Analysis:
Average Suffix Length: 1.250768024537336
Top 10 suffixes: [('s', 55539), ('e', 35090), ('t', 19226), ('on', 14811), ('y', 14792), ('er', 8663), ('d', 7965), ('m', 4355), ('ing', 3869), ('k', 3658)]
